# US-Ignite Portland ETL Notebook

## Portland Datasets

Portland:

Building FootPrints: http://gis-pdx.opendata.arcgis.com/datasets/building-footprints
GeoJson: https://opendata.arcgis.com/datasets/1078959662ce49cca059fe2f8930c194_48.geojson

Traffic Counts: http://gis-pdx.opendata.arcgis.com/datasets/traffic-volume-counts
GeoJson: https://opendata.arcgis.com/datasets/6ba5258ffea34e878168ddc8cf34f7e3_250.geojson

Buildings Elevations: http://gis-pdx.opendata.arcgis.com/datasets/935e774546cf4383a881a9c12c182e6e_184
GeoJson: https://opendata.arcgis.com/datasets/935e774546cf4383a881a9c12c182e6e_184.geojson

Residential Building Permits: http://gis-pdx.opendata.arcgis.com/datasets/797a38ddb60a471392ea9134448ad6c2_89
GeoJson: https://opendata.arcgis.com/datasets/797a38ddb60a471392ea9134448ad6c2_89.geojson

Busineess Districts: http://gis-pdx.opendata.arcgis.com/datasets/65c53b0f616a4a13b51e40c237c70050_11
GeoJson: https://opendata.arcgis.com/datasets/935e774546cf4383a881a9c12c182e6e_184.geojson

Recommended Bike Routes: http://gis-pdx.opendata.arcgis.com/datasets/recommended-bicycle-routes
GeoJson: https://opendata.arcgis.com/datasets/fcd1f0403cbf484bb46233b44d96f220_183.geojson

Transit stop peak arrival: http://gis-pdx.opendata.arcgis.com/datasets/transit-stop-peak-arrival?selectedAttribute=PMPeakArrivals
GeoJson: https://opendata.arcgis.com/datasets/9dbb8fa9235d4ac796065787599ab4ca_251.geojson

In [1]:
#import libraries from Carto.
from cartoframes.auth import set_default_credentials, Credentials
from cartoframes.viz import Map, Layer
from cartoframes.data import Dataset
from cartoframes.viz.helpers import color_continuous_layer
import pandas as pd
import geopandas as gpd
import fiona

#set_default_credentials(
 #   base_url='https://usignite.carto.com/',
  #  api_key='nJor0dLGngHDZnlUwqK1Ow'
#)

In [2]:
#load .env with credentials
from dotenv import load_dotenv
load_dotenv()
import os
BASE_URL = os.getenv("BASE_URL")
API_KEY = os.getenv("API_KEY")
USERNAME = os.getenv("USERNAME")

set_default_credentials(
    base_url=BASE_URL,
    api_key=API_KEY
)

credentials = Credentials(USERNAME, API_KEY)

In [3]:
print(API_KEY)

81c922dc336bd82103d92a768816a4567c444909


## Extract: Bring GeoJson into Notebook

In [4]:
url="https://opendata.arcgis.com/datasets/9dbb8fa9235d4ac796065787599ab4ca_251.geojson"


In [5]:
transit_stop_peak_arrival_gpdf = gpd.read_file(url)

## Transform
Add here some code to do data transformation (remove outliers, null values, etc.). It's probably best to do this type of work on the geopandas dataframe 

In [6]:
transit_stop_peak_arrival_gpdf.head()

,OBJECTID,StopID,AMPeakArrivals,PMPeakArrivals,MeetsStandard,InBufferArea,geometry
0,1,20,3,4,No,No,POINT (-122.5697882371598 45.54267518056737)
1,2,21,3,4,No,No,POINT (-122.5700462503774 45.54315785817379)
2,3,25,3,4,No,No,POINT (-122.5690016300762 45.54815704386191)
3,4,26,3,4,No,No,POINT (-122.5700489363401 45.54566103724926)
4,5,40,8,13,Yes,Yes,POINT (-122.711707188526 45.58857758435087)


## Load: Upload to CARTO

In [7]:
transit_stop_peak_arrival_d = Dataset(transit_stop_peak_arrival_gpdf)
transit_stop_peak_arrival_d.upload(table_name='portland_transit_stop_peak_arrival', if_exists='replace', credentials=credentials)

The following columns were changed in the CARTO copy of this dataframe:
OBJECTID -> objectid
StopID -> stopid
AMPeakArrivals -> ampeakarrivals
PMPeakArrivals -> pmpeakarrivals
MeetsStandard -> meetsstandard
InBufferArea -> inbufferarea


/home/pui-user/anaconda3/lib/python3.6/site-packages/carto/sql.py:229: UserWarning: Batch SQL job created with job_id: 4aa76968-12e6-4057-a2ce-1d8c5870419c
  warnings.warn('Batch SQL job created with job_id: {job_id}'.format(job_id=data['job_id']))


## Make a quick map using one of CARTO's helper functions

In [8]:
transit_stop_peak_arrival_CARTO = Dataset('portland_transit_stop_peak_arrival')
Map(
    color_continuous_layer(transit_stop_peak_arrival_CARTO, 'pmpeakarrivals','PM Peak Arrivals')
)